# Cell Geometry Studies
The purpose of this notebook is to use the cell geometry root file to
perform investigations on average locations of cells in the calorimeter
layers. This is for the purpose of track projections for now. For 
calorimeter layers *parallel* to the beam line, we seek a common
*'fixed rPerp'*. For layers *perpendicular* to the beam line we seek a
common *'fixed z'*.


**Contributors:**

Russell Bate - 
russellbate@phas.ubc.ca

#### Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uproot as ur
import awkward as ak
import time as t
from time import perf_counter as cput

print("Awkward version: "+str(ak.__version__))
print("Uproot version: "+str(ur.__version__))

geo_rootfile_loc = '/fast_scratch/atlas_images/v01-45/cell_geo.root'

Awkward version: 1.5.1
Uproot version: 4.1.8


#### Data and Branches

In [2]:
track_branches = ['trackEta_EMB1', 'trackPhi_EMB1', 'trackEta_EMB2', 'trackPhi_EMB2', 'trackEta_EMB3', 'trackPhi_EMB3',
                  'trackEta_TileBar0', 'trackPhi_TileBar0', 'trackEta_TileBar1', 'trackPhi_TileBar1',
                  'trackEta_TileBar2', 'trackPhi_TileBar2']

geo_branches = ["cell_geo_ID", "cell_geo_eta", "cell_geo_phi", "cell_geo_rPerp", "cell_geo_sampling"]


calo_layers = ['EMB1', 'EMB2', 'EMB3', 'EME1', 'EME2', 'EME3', 'HEC0', 'HEC1',
    'HEC2', 'HEC3', 'TileBar0', 'TileBar1', 'TileBar2', 'TileGap1', 'TileGap2',
    'TileGap3', 'TileExt0', 'TileExt1', 'TileExt2']
calo_numbers = set([1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])

fixed_z_layers = ['EME1', 'EME2', 'EME3', 'HEC0', 'HEC1', 'HEC2', 'HEC3',
    'HEC0', 'HEC1', 'HEC2', 'HEC3']
fixed_z_numbers = set([5,6,7,8,9,10,11])

fixed_r_layers = ['EMB1', 'EMB2', 'EMB3', 'EME1', 'EME2', 'EME3', 'TileBar0',
    'TileBar1', 'TileBar2', 'TileGap1', 'TileGap2', 'TileGap3', 'TileExt0',
    'TileExt1', 'TileExt2']
fixed_r_numbers = set([1,2,3,12,13,14,15,16,17,18,19,20])


calo_dict = dict(zip(calo_numbers, calo_layers))

for item in calo_dict.items():
    print(item)

(1, 'EMB1')
(2, 'EMB2')
(3, 'EMB3')
(5, 'EME1')
(6, 'EME2')
(7, 'EME3')
(8, 'HEC0')
(9, 'HEC1')
(10, 'HEC2')
(11, 'HEC3')
(12, 'TileBar0')
(13, 'TileBar1')
(14, 'TileBar2')
(15, 'TileGap1')
(16, 'TileGap2')
(17, 'TileGap3')
(18, 'TileExt0')
(19, 'TileExt1')
(20, 'TileExt2')


## Load Geometry Tree

In [3]:
geo_file = ur.open(geo_rootfile_loc)
CellGeo_tree = geo_file["CellGeo"]
CellGeo_tree.show(interpretation_width=30, name_width=25)

name                      | typename                 | interpretation                
--------------------------+--------------------------+-------------------------------
cell_geo_ID               | std::vector<uint64_t>    | AsJagged(AsDtype('>u8'), he...
cell_geo_sampling         | std::vector<uint16_t>    | AsJagged(AsDtype('>u2'), he...
cell_geo_eta              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_phi              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_rPerp            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_deta             | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_dphi             | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_volume           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_sigma            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
cell_geo_prevInPhi        | std::vector<int32_t>     |

In [4]:
cell_geo_sampling = CellGeo_tree.arrays(filter_name='cell_geo_sampling')\
    ['cell_geo_sampling'].to_numpy().flatten()
print(cell_geo_sampling.shape)
print(cell_geo_sampling[:10])

(187650,)
[6 6 6 6 6 6 6 6 6 6]


### Load Events

In [5]:
CGS = CellGeo_tree.arrays(filter_name='cell_geo_sampling')\
    ['cell_geo_sampling'].to_numpy().flatten()
CGE = CellGeo_tree.arrays(filter_name='cell_geo_eta')\
    ['cell_geo_eta'].to_numpy().flatten()
CGP = CellGeo_tree.arrays(filter_name='cell_geo_phi')\
    ['cell_geo_phi'].to_numpy().flatten()
CGRp = CellGeo_tree.arrays(filter_name='cell_geo_rPerp')\
    ['cell_geo_rPerp'].to_numpy().flatten()

### Find X, Y, Z

In [6]:
def xyz(eta, phi, rperp):
    theta = 2*np.arctan( np.exp(-eta) )

    cell_x = rperp*np.cos(phi)
    cell_y = rperp*np.sin(phi)
    cell_z = rperp/np.tan(theta)
    
    return cell_x, cell_y, cell_z

In [7]:
X, Y, Z = xyz(eta=CGE, phi=CGP, rperp=CGRp)

# this is because of symmetry in the calo geometry of course
Z = np.abs(Z)

### Use Masks to Find Averages

In [8]:
z_av_dict = dict()
r_av_dict = dict()

for num in fixed_z_numbers:
    mask = CGS == num
    mean = np.mean(Z[mask])
    z_av_dict[str(num)] = mean
    
for num in fixed_r_numbers:
    mask = CGS == num
    mean = np.mean(CGRp[mask])
    r_av_dict[str(num)] = mean

print('-- Fixed Z values --')
for i, (key, value) in enumerate(z_av_dict.items()):
    print('{} : {:f}  --  {:s}'.format(key, value, calo_dict[int(key)]))

print()
print('-- Fixed rPerp values --')
for i, (key, value) in enumerate(r_av_dict.items()):
    print('{} : {:f} --  {:s}'.format(key, value, calo_dict[int(key)]))

-- Fixed Z values --
5 : 3790.032471  --  EME1
6 : 3983.684326  --  EME2
7 : 4195.843750  --  EME3
8 : 4461.250000  --  HEC0
9 : 4869.500000  --  HEC1
10 : 5424.500000  --  HEC2
11 : 5905.000977  --  HEC3

-- Fixed rPerp values --
1 : 1532.177246 --  EMB1
2 : 1723.890869 --  EMB2
3 : 1923.020996 --  EMB3
12 : 2450.000000 --  TileBar0
13 : 2995.000000 --  TileBar1
14 : 3630.000000 --  TileBar2
15 : 3215.000000 --  TileGap1
16 : 3630.000000 --  TileGap2
17 : 2246.500000 --  TileGap3
18 : 2450.000000 --  TileExt0
19 : 2870.000000 --  TileExt1
20 : 3480.000000 --  TileExt2
